In [1]:
from fastapi import FastAPI, UploadFile, File, HTTPException
import pandas as pd
from io import BytesIO
from fastapi import FastAPI, UploadFile, File, HTTPException
from pydantic import BaseModel
from typing import Dict, List
from jinja2 import Template, Environment, meta

In [2]:
app = FastAPI(title="AI Email Agent API")

csv upload

In [3]:


# Your existing CSV loader function (adapted)
def load_csv_file(file) -> pd.DataFrame:
    try:
        df = pd.read_csv(file)
    except Exception as e:
        raise HTTPException(status_code=400, detail=f"Error reading CSV: {e}")

    df.columns = [c.strip().lower() for c in df.columns]

    if "email" not in df.columns:
        raise HTTPException(status_code=400, detail="CSV is missing required column: 'email'")

    for col in df.columns:
        if df[col].dtype == object:
            df[col] = df[col].astype(str).str.strip()

    df = df[df["email"].notna() & (df["email"] != "")]
    df = df.drop_duplicates(subset=["email"])

    return df

# FastAPI endpoint
@app.post("/upload_csv")
async def upload_csv(file: UploadFile = File(...)):
    if not file.filename.endswith(".csv"):
        raise HTTPException(status_code=400, detail="File must be a CSV.")

    contents = await file.read()
    df = load_csv_file(BytesIO(contents))

    preview = df.head().to_dict(orient="records")
    total_rows = len(df)

    return {"preview": preview, "total_rows": total_rows}

email validate

In [4]:
# Function to validate DataFrame rows (reusing the previous logic)
def validate_recipients_df(df: pd.DataFrame):
    valid_rows = []
    invalid_rows = []

    for idx, row in df.iterrows():
        try:
            recipient = RecipientRow(**row.to_dict())
            valid_rows.append(recipient.dict())  # convert to dict for JSON response
        except ValidationError as e:
            invalid_rows.append({
                "row_index": idx,
                "errors": e.errors(),
                "data": row.to_dict()
            })

    return valid_rows, invalid_rows

# FastAPI endpoint to validate uploaded CSV
@app.post("/validate_recipients")
async def validate_recipients(file: UploadFile = File(...)):
    if not file.filename.endswith(".csv"):
        raise HTTPException(status_code=400, detail="File must be a CSV.")

    contents = await file.read()
    df = load_csv_file(BytesIO(contents))  # reuse CSV loader from previous endpoint

    valid_rows, invalid_rows = validate_recipients_df(df)

    return {
        "valid_count": len(valid_rows),
        "invalid_count": len(invalid_rows),
        "valid_rows": valid_rows,
        "invalid_rows": invalid_rows
    }


template for personalized 

In [5]:



# Pydantic model for incoming request
class TemplateRequest(BaseModel):
    template_str: str
    recipients: List[Dict]  # list of recipient dicts

# Function to render a single recipient
def render_email_template(template_str: str, recipient: dict):
    env = Environment()
    template = env.from_string(template_str)
    
    # Check placeholders
    ast = env.parse(template_str)
    placeholders = meta.find_undeclared_variables(ast)
    missing = [ph for ph in placeholders if ph not in recipient]
    if missing:
        raise ValueError(f"Missing placeholder(s) in recipient data: {missing}")

    return template.render(**recipient)

# FastAPI endpoint to render template(s)
@app.post("/render_template")
async def render_template(request: TemplateRequest):
    rendered_results = []
    errors = []

    for idx, recipient in enumerate(request.recipients):
        try:
            rendered_body = render_email_template(request.template_str, recipient)
            rendered_results.append({
                "recipient_index": idx,
                "email": recipient.get("email"),
                "rendered_body": rendered_body
            })
        except ValueError as e:
            errors.append({
                "recipient_index": idx,
                "email": recipient.get("email"),
                "error": str(e)
            })

    return {"rendered": rendered_results, "errors": errors}


In [6]:

# Request model
class GenerateEmailsRequest(BaseModel):
    template_str: str
    recipients: List[Dict]
    mode: str = "personalized"  # "personalized" or "single"

# Endpoint to generate emails based on personalization mode
@app.post("/generate_emails")
async def generate_emails(request: GenerateEmailsRequest):
    if request.mode not in ["personalized", "single"]:
        raise HTTPException(status_code=400, detail="Mode must be 'personalized' or 'single'")

    rendered_emails = []

    try:
        if request.mode == "single":
            context = request.recipients[0] if request.recipients else {}
            body = render_email_template(request.template_str, context)
            for recipient in request.recipients:
                rendered_emails.append({
                    "email": recipient.get("email"),
                    "rendered_body": body
                })
        else:  # personalized
            for recipient in request.recipients:
                body = render_email_template(request.template_str, recipient)
                rendered_emails.append({
                    "email": recipient.get("email"),
                    "rendered_body": body
                })
    except ValueError as e:
        raise HTTPException(status_code=400, detail=str(e))

    return {"rendered_emails": rendered_emails}


In [1]:
from fastapi import FastAPI
from pydantic import BaseModel
import asyncio

# 👉 import your exact functions without modification
from your_agent_file import run_graph   
# (replace your_agent_file with the actual filename)


app = FastAPI()


# -----------------------------
# REQUEST MODELS
# -----------------------------

class GenerateRequest(BaseModel):
    user_message: str
    history: list  # [{"role": "...", "content": "..."}]


class SendEmailRequest(BaseModel):
    email_content: str
    recipients: list[str]


# -----------------------------
# 1️⃣ GENERATE EMAIL ENDPOINT
# -----------------------------

@app.post("/generate")
async def generate(req: GenerateRequest):
    # directly run your graph, no changes
    response = await run_graph(req.user_message, req.history)
    return {"email": response}


# -----------------------------
# 2️⃣ SEND EMAIL ENDPOINT
# -----------------------------

@app.post("/send")
async def send(req: SendEmailRequest):

    # ⭐ DO NOT CHANGE ANYTHING IN YOUR EMAIL SYSTEM
    # just wrap it here

    sent = []

    for r in req.recipients:
        # call your existing send logic here
        sent.append(r)

    return {
        "status": "success",
        "sent_to": sent,
        "email_content": req.email_content
    }


ModuleNotFoundError: No module named 'agent'